In [18]:
### create a data class
class Sentiment:
    negative = "negative"
    neutral = "neutral"
    positive = "positive"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.negative
        elif self.score == 3:
            return Sentiment.neutral
        else: #score of 4 or 5
            return Sentiment.positive



## LOAD DATA

In [17]:
import json
import os

file_name = 'data/books_small.json'


### assign review to read a line in json file
### assign empty list to reviews
### append all reviewText and overall score to reviews
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))


reviews[5].text


'Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!'

## PREP DATA

In [20]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42) 

In [26]:
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

train_x[0]
train_y[0]

'positive'

### bag of words vectorization